In [24]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.patches  import Circle   
from scipy.optimize import least_squares
import open3d as o3d
import sys
import os
import time

In [10]:
def viz(pcd, name="3D Point Cloud"):
    o3d.visualization.draw_geometries([pcd], 
                                  window_name=name,
                                  width=1280,
                                  height=1080)

In [11]:
def compute_fpfh_features(pcd, feature_radius):
    """
    Вычисляет FPFH дескрипторы для облака точек.
    """
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamHybrid(radius=feature_radius, max_nn=50)) # max_nn для FPFH обычно больше
    return pcd_fpfh

In [15]:
# Кропаем по Z каждый pcd, достаем пластину

pcd = o3d.io.read_point_cloud("file_stitching/crop_to_crop_testing/output/fragment_4.pcd")
pcd = pcd.remove_non_finite_points()

print(f"Исходное количество точек: {len(pcd.points)}")

# Получаем текущие границы облака
min_overall_bound = pcd.get_min_bound()
max_overall_bound = pcd.get_max_bound()

print(f"Исходные минимальные координаты: {min_overall_bound}")
print(f"Исходные максимальные координаты: {max_overall_bound}")


z_max_to_keep = max_overall_bound[2]
z_min_to_keep = max_overall_bound[2] - 150


min_bound_for_crop = np.array([min_overall_bound[0], min_overall_bound[1], z_min_to_keep])
max_bound_for_crop = np.array([max_overall_bound[0], max_overall_bound[1], z_max_to_keep])


# Создаем выровненный по осям ограничивающий ящик
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound_for_crop, max_bound_for_crop)

# 3. Отсечение Облака Точек
cropped_pcd = pcd.crop(bbox)

print(f"Количество точек после отсечения по Z: {len(cropped_pcd.points)}")


# 4. Сохранение Отсеченного Облака Точек
output_filename = "file_stitching/crop_to_crop_testing/output/cloud_recon_4_cropped_.pcd" # Имя выходного файла
o3d.io.write_point_cloud(output_filename, cropped_pcd) # , write_ascii=True)

print(f"Отсеченное облако точек сохранено в: {output_filename}")

# Визуализация исходного и отсеченного облаков (для сравнения)
viz(pcd, "Исходное облако")
viz(cropped_pcd, "Отсеченное по Z облако")


Исходное количество точек: 291399
Исходные минимальные координаты: [0. 0. 0.]
Исходные максимальные координаты: [300.58532715 150.76516724  68.1679306 ]
Количество точек после отсечения по Z: 291399
Отсеченное облако точек сохранено в: file_stitching/crop_to_crop_testing/output/cloud_recon_4_cropped_.pcd


In [35]:
pcd_1 = o3d.io.read_point_cloud(Path("file_stitching/crop_to_crop_testing/output/cloud_recon_2_cropped_.pcd"))
pcd_2 = o3d.io.read_point_cloud(Path("file_stitching/crop_to_crop_testing/output/cloud_recon_4_cropped_.pcd"))


viz(pcd_1)
viz(pcd_2)


In [36]:
voxel_size = 1.5

downsampled_pcd_1 = pcd_1.voxel_down_sample(voxel_size=voxel_size)
downsampled_pcd_2 = pcd_2.voxel_down_sample(voxel_size=voxel_size)


print(f"Исходное количество точек: {len(pcd_1.points)}")
print(f"Количество точек после downsampling: {len(downsampled_pcd_1.points)}")
print(f"Исходное количество точек: {len(pcd_2.points)}")
print(f"Количество точек после downsampling: {len(downsampled_pcd_2.points)}")


viz(downsampled_pcd_1)
viz(downsampled_pcd_2)



Исходное количество точек: 286507
Количество точек после downsampling: 25550
Исходное количество точек: 291399
Количество точек после downsampling: 25415


In [37]:
downsampled_pcd_norm_1 = downsampled_pcd_1
downsampled_pcd_norm_2 = downsampled_pcd_2


search_radius = voxel_size * 2  # Пример: 30 мм, если voxel_size ~10 мм
max_num_neighbors = 20 # Максимальное количество соседей

print(f"Вычисляем нормали для облака с {len(downsampled_pcd_norm_1.points)} точками...")
print(f"Вычисляем нормали для облака с {len(downsampled_pcd_norm_2.points)} точками...")




# Вычисление нормалей
downsampled_pcd_norm_1.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=search_radius, max_nn=max_num_neighbors)
)
downsampled_pcd_norm_2.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=search_radius, max_nn=max_num_neighbors)
)

print("Нормали вычислены.")

# Опционально: Ориентация нормалей (для согласованности)
# Это может улучшить качество некоторых последующих операций.
# Для вашей калибровочной панели, если она относительно плоская,
# `orient_normals_consistent_tangent_plane` может быть хорошим выбором.
downsampled_pcd_norm_1.orient_normals_consistent_tangent_plane(k=max_num_neighbors)
downsampled_pcd_norm_2.orient_normals_consistent_tangent_plane(k=max_num_neighbors)

print("Нормали ориентированы.")

# Визуализация нормалей (ОЧЕНЬ РЕКОМЕНДУЕТСЯ для проверки!)
# Визуализация может быть медленной для очень большого количества точек с нормалями.
o3d.visualization.draw_geometries([downsampled_pcd_norm_1], point_show_normal=True, width=1280, height=1080)
o3d.visualization.draw_geometries([downsampled_pcd_norm_2], point_show_normal=True, width=1280, height=1080)


Вычисляем нормали для облака с 25550 точками...
Вычисляем нормали для облака с 25415 точками...
Нормали вычислены.
Нормали ориентированы.


In [38]:
o3d.visualization.draw_geometries([downsampled_pcd_norm_1], point_show_normal=True, width=1280, height=1080)
o3d.visualization.draw_geometries([downsampled_pcd_norm_2], point_show_normal=True, width=1280, height=1080)


In [39]:
feature_radius = search_radius * 5

fpfh_side1 = compute_fpfh_features(downsampled_pcd_norm_1, feature_radius)
fpfh_side2 = compute_fpfh_features(downsampled_pcd_norm_2, feature_radius)


print("FPFH дескрипторы вычислены для всех облаков.")

FPFH дескрипторы вычислены для всех облаков.


In [40]:
def execute_icp_registration(source_pcd, target_pcd, init_transform, voxel_size, max_iterations=200):
    """
    Выполняет точную регистрацию ICP (Point-to-Plane).
    """
    # max_correspondence_distance для ICP обычно равен или чуть больше voxel_size
    icp_distance_threshold = voxel_size * 1.5 # Или voxel_size * 1.0, 2.0. Поэкспериментируйте.

    print(f"ICP: Max correspondence distance set to {icp_distance_threshold:.2f} (mm if voxel_size is mm)")

    reg_result = o3d.pipelines.registration.registration_icp(
        source_pcd,
        target_pcd,
        icp_distance_threshold, # Порог расстояния для ICP
        init_transform,         # Начальная трансформация (от глобальной регистрации)
        o3d.pipelines.registration.TransformationEstimationPointToPlane(), # Point-to-Plane ICP
        o3d.pipelines.registration.ICPConvergenceCriteria(
            relative_fitness=1e-6,  # Порог изменения фитнеса (очень маленький)
            relative_rmse=1e-6,     # Порог изменения RMSE (очень маленький)
            max_iteration=max_iterations # Максимальное количество итераций ICP
        )
    )
    return reg_result

In [42]:
# Функция для выполнения глобальной регистрации между двумя облаками
def execute_global_registration(source_pcd, target_pcd, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5 # или 2.0. Допустимый порог расстояния для соответствий

    # Параметры RANSAC для глобальной регистрации
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_pcd, target_pcd, source_fpfh, target_fpfh,
        mutual_filter=True, 
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n=3,
        #check_overlap=False, # Можно поставить True, если облака не должны сильно перекрываться, но для вашей задачи False лучше
        # check_fitness=True, check_inlier_rmse=True, # Дополнительные проверки, могут замедлить
        # Исправлено: max_iteration теперь часть criteria
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(
            max_iteration=3000, # Максимальное количество итераций
            confidence=0.99   # Уровень уверенности
        )
    )
    return result

In [43]:
print("Выполняем глобальную регистрацию: pcd_side1 -> pcd_front")


reg_side1_to_front = execute_global_registration(downsampled_pcd_norm_2, downsampled_pcd_norm_1, fpfh_side2, fpfh_side1, voxel_size)

print(f"Результат глобальной регистрации (pcd_side1 -> pcd_front):")
print(reg_side1_to_front)


initial_transform_side1 = reg_side1_to_front.transformation


pcd_side1_aligned = downsampled_pcd_norm_2.transform(initial_transform_side1)

print("Визуализация после глобальной регистрации (1 к 3)...")
o3d.visualization.draw_geometries(
    [downsampled_pcd_norm_1, pcd_side1_aligned],
    window_name="Глобальное выравнивание: 1 к 3",
    width=1280, height=1080
)


# 3. Точная регистрация ICP: downsampled_pcd_norm_1 -> downsampled_pcd_norm_3 (Front)
print("\n--- Точная регистрация ICP: downsampled_pcd_norm_1 -> downsampled_pcd_norm_3 ---")
reg_icp_1_to_3 = execute_icp_registration(
    pcd_side1_aligned, # Используем уже грубо выровненное облако как источник для ICP
    downsampled_pcd_norm_1, # Целевое облако
    initial_transform_side1, # Начальная трансформация для ICP (от глобальной регистрации)
    voxel_size
)

print(f"Результат ICP (fitness={reg_icp_1_to_3.fitness:.4f}, rmse={reg_icp_1_to_3.inlier_rmse:.4f}):")
print(reg_icp_1_to_3)


final_transform_1_to_3 = reg_icp_1_to_3.transformation
pcd_1_final_aligned = downsampled_pcd_norm_2.transform(final_transform_1_to_3)

# --- Визуализация после ICP (для проверки точности) ---
print("Визуализация после ICP (1 к 3)...")
o3d.visualization.draw_geometries(
    [downsampled_pcd_norm_1, pcd_1_final_aligned],
    window_name="ICP выравнивание: 1 к 3",
    width=1280, height=1080
)



# --- Объединение двух выровненных облаков ---
print("\n--- Объединение выровненных облаков ---")
combined_pcd_4_clouds = o3d.geometry.PointCloud()

# Собираем точки из обоих облаков
# downsampled_pcd_norm_3.points и pcd_1_final_aligned.points
# уже являются o3d.utility.Vector3dVector.
# Чтобы их объединить, сначала преобразуем в NumPy массивы, объединим,
# а затем обратно в Vector3dVector.
combined_pcd_4_clouds.points = o3d.utility.Vector3dVector(
    np.concatenate((
        np.asarray(downsampled_pcd_norm_1.points),
        np.asarray(pcd_1_final_aligned.points)
    ))
)

# Собираем нормали из обоих облаков
combined_pcd_4_clouds.normals = o3d.utility.Vector3dVector(
    np.concatenate((
        np.asarray(downsampled_pcd_norm_1.normals),
        np.asarray(pcd_1_final_aligned.normals)
    ))
)

Выполняем глобальную регистрацию: pcd_side1 -> pcd_front
Результат глобальной регистрации (pcd_side1 -> pcd_front):
RegistrationResult with fitness=8.629549e-01, inlier_rmse=8.562587e-01, and correspondence_set size of 21932
Access transformation to get result.
Визуализация после глобальной регистрации (1 к 3)...

--- Точная регистрация ICP: downsampled_pcd_norm_1 -> downsampled_pcd_norm_3 ---
ICP: Max correspondence distance set to 2.25 (mm if voxel_size is mm)
Результат ICP (fitness=0.5981, rmse=0.6546):
RegistrationResult with fitness=5.981114e-01, inlier_rmse=6.545530e-01, and correspondence_set size of 15201
Access transformation to get result.
Визуализация после ICP (1 к 3)...

--- Объединение выровненных облаков ---


In [44]:
o3d.visualization.draw_geometries([downsampled_pcd_norm_2, combined_pcd_4_clouds], 
                                  window_name="Глобальное выравнивание: Side1 к Front",
                                  width=1280,
                                  height=1080)

In [45]:
viz(downsampled_pcd_norm_2)
viz(downsampled_pcd_norm_1)
viz(combined_pcd_4_clouds)

In [47]:
# --- Опционально: Финальный downsample объединенного облака ---
# Это поможет уменьшить плотность в перекрывающихся областях и убрать небольшие шумы.
# Используйте меньший voxel_size для этой последней чистки.
print(f"Точек до финального downsample (объединенное): {len(combined_pcd_4_clouds.points)}")
combined_pcd_4_clouds = combined_pcd_4_clouds.voxel_down_sample(voxel_size=voxel_size / 2)
print(f"Точек после финального downsample (объединенное): {len(combined_pcd_4_clouds.points)}")

# --- Визуализация итогового объединенного облака ---
print("\n--- Финальная визуализация объединенного облака (две пластины) ---")
o3d.visualization.draw_geometries(
    [combined_pcd_4_clouds],
    zoom=0.8,
    front=[0.45, -0.9, 0.0],
    lookat=[0.0, 0.0, 0.0],
    up=[0.0, 0.0, 1.0],
    window_name="Объединенное облако двух пластин"
)

# --- Сохранение итогового объединенного облака (опционально) ---
output_path = "combined_two_clouds.pcd"
o3d.io.write_point_cloud(output_path, combined_pcd_4_clouds)
print(f"Объединенное облако сохранено в: {output_path}")




o3d.visualization.draw_geometries([downsampled_pcd_norm_1, pcd_side1_aligned], 
                                  window_name="Глобальное выравнивание: Side1 к Front",
                                  width=1280,
                                  height=1080)

Точек до финального downsample (объединенное): 45779
Точек после финального downsample (объединенное): 45779

--- Финальная визуализация объединенного облака (две пластины) ---
Объединенное облако сохранено в: combined_two_clouds.pcd
